# Interactions with EdgeX Foundry Microservices

In this notebook, we will explore the functionality of EdgeX Foundy. EdgeX Foundry is a microservice oriented architecture that enables programatic integration of IoT devices on an edge network. EdgeX is specifically build with industrial and business use-cases in mind.

## Install the Python Requests Module

In [ ]:
import pip
def install(package):
    if hasattr(pip, 'main'):
        pip.main(['install', package])
    else:
        pip._internal.main(['install', package])

pip.main(['install', 'pip'])

## API Reference
As we go through this lab, you will find it useful to refer to the API reference page.
https://docs.edgexfoundry.org/Ch-APIReference.html

## Loading the Requests Library

In [ ]:
# Import the Python HTTP Requests Library
import requests

After loading the library, we can make our first HTTP request to demonstrate how to use the Python3 Requests library

In [ ]:
r = requests.get('https://api.github.com/orgs/SSG-DRD-IOT/events')

In [ ]:
# View the Python Response Object
r

In [ ]:
# Get the HTTP status code that was returned with the request
r.status_code

In [ ]:
# Test whether the status code was 200 OK, the HTTP success code.
r.status_code == requests.codes.ok

In [ ]:
print(r.text)

In [ ]:
import json
def pjson(text):
    parsed = json.loads(text)
    print(json.dumps(parsed, indent=4, sort_keys=True))
    
pjson(r.text)

# Provision a Modbus Device
## Modbus Device Documentation

Manufacturer's Data Sheet: http://www.datanab.com/zc/docs/sensors/MBus_RTH_LCD.pdf

### Important Settings

RS485 Port Configuration

* Modbus - Device communication settings
* Modbus device Baud rate = 19200
* Modbus device Parity = N (none)
* Modbus device Data bits = 8
* Modbus device Stop bits = 1 
* Modbus device address: 254 (device specific) 
* Modbus device register: 100, 101, 102 (device specific) 
* Modbus device register data type: Integer/Floating point (device specific)

Registers 100, 101 and 102 will have the **temperature in F**, the **temperature in C** and the **humidity**.


### IEI Tank Settings
* Port = /dev/ttyUSB0
* Port pin 1 = DATA +
* Port pin 2 = DATA -
* Port pin 3 = GND


## Modbus Device Definitation in YAML Format

### Modify the following fields
* name <– A/a ~Z/z and 0 ~ 9 && this will be needed in the future
* manufacturer <– A/a ~Z/z and 0 ~ 9
* model <– A/a ~Z/z and 0 ~ 9
* description <– A/a ~Z/z and 0 ~ 9
* labels <– A/a ~Z/z and 0 ~ 9

#### **DeviceResources**
* name: <– A/a ~Z/z and 0 ~ 9
* description: <– A/a ~Z/z and 0 ~ 9
* attributes: only edit the text inside the parenthesis
* value: only edit the text inside the parenthesis
* units: only edit the text inside the parenthesis

#### **Resources**
* name: <– A/a ~Z/z and 0 ~ 9
* get : only edit the text inside the parenthesis
* set: only edit the text inside the parenthesis

#### **Commands**
* name: <– A/a ~Z/z and 0 ~ 9
* path: “/api/v1/device/{deviceId}/OnlyEditThisWord” <– A/a ~Z/z and 0 ~ 9
* Code ”200” - expectedvalues: [make same as OnlyEditThisWord]
* Code ”500” - Do not edit this section



### The YAML File

In [ ]:
name: "MBUS_RTH_LCD"
manufacturer: "DATANAB"
model: "MBUS_RTH_LCD"
labels: 
 - "temperature"
 - "modbus"
 - "industrial"
description: "Modbus Enabled Room Temperature/Humididty Sensor with LCD"

deviceResources:
      -
        name: "TemperatureDegF"
        description: "Room Temperature in Degrees Fahrenheit."
        attributes:
            { HoldingRegister: "100" }
        properties:
            value:
                { type: "Float", readWrite: "R", size: "1", scale: "0.1", minimum: "40.0", maximum: "122.0", defaultValue: "1"}
            units:
                { type: "String", readWrite: "R", defaultValue: "degrees fahrenheit"}
      -
        name: "TemperatureDegC"
        description: "Room Temperature in Degrees Celsius."
        attributes:
            { HoldingRegister: "101" }
        properties:
            value:
                { type: "Float", size: "1", scale: "0.1", readWrite: "R", minimum: "5.0", maximum: "50.0", defaultValue: "1"}
            units:
                { type: "String", readWrite: "R", defaultValue: "degrees celsius"}
      -
        name: "HumidityPercentRH"
        description: "Room Humidity in %RH."
        attributes:
            { HoldingRegister: "102" }
        properties:
            value:
                { type: "Float", size: "1", scale: "0.1", readWrite: "R", minimum: "0.0", maximum: "95", defaultValue: "1"}
            units:
                { type: "String", readWrite: "R", defaultValue: "%RH"}

resources:
      -
        name: "TemperatureDegF"
        get:
          - { index: "1", operation: "get", object: "TemperatureDegF", parameter: "TemperatureDegF", property: "value" }
      -
        name: "TemperatureDegC"
        get:
          - { index: "1", operation: "get", object: "TemperatureDegC", parameter: "TemperatureDegC", property: "value" }
      -
        name: "HumidityPercentRH"
        get:
          - { index: "1", operation: "get", object: "HumidityPercentRH", parameter: "HumidityPercentRH", property: "value" }

commands:
      -
        name: "TemperatureDegF"
        get:
            path: "/api/v1/device/{deviceId}/TemperatureDegF"
            responses: 
              -
                code: "200"
                description: "Get the temperature in degrees F"
                expectedValues: ["TemperatureDegF"]
              -
                code: "503"
                description: "service unavailable"
                expectedValues: []
      -
        name: "TemperatureDegC"
        get:
            path: "/api/v1/device/{deviceId}/TemperatureDegC"
            responses: 
              -
                code: "200"
                description: "Get the temperature in degrees C"
                expectedValues: ["TemperatureDegC"]
              -
                code: "503"
                description: "service unavailable"
                expectedValues: []
      -
        name: "HumidityPercentRH"
        get:
            path: "/api/v1/device/{deviceId}/HumidityPercentRH"
            responses: 
              -
                code: "200"
                description: "Get the humidity in %RH"
                expectedValues: ["HumidityPercentRH"]
              -
                code: "503"
                description: "service unavailable"
                expectedValues: []





### Configure the Docker-Compose.yaml Startup Script

Now that we have a configuration file for the modbus device we need to alter the startup script to enable the Edge X modbus service.

Open up the docker-compose.yaml file
And uncomment the lines that enable the Modbus Service.

In [ ]:
device-modbus:
    image: edgexfoundry/docker-device-modbus-go:0.7.1
    ports:
        - "49991:49991"
    container_name: edgex-device-modbus
    hostname: edgex-device-modbus
    networks:
        - edgex-network
    volumes:
        - db-data:/data/db
        - log-data:/edgex/logs
        - consul-config:/consul/config
        - consul-data:/consul/data
    depends_on:
        - data
        - command


### Start the EdgeX Services

In [ ]:
#!/bin/bash
sudo docker-compose up

### Verify that the Modbus Service Started

In [ ]:
!docker-compose ps

# Postman

Postman is a tool that helps developers send restful HTTP and HTTPS requests. EdgeX is entirely configured through restful HTTP requests which makes postman a very useful tool for interacting with EdgeX.

In the repository for this lab you should see a number of JSON files. These files contain all the API for interacting with Edge X and can be imported into postman. 

<img src="images/postman.png"/>


## Create an Addressable

The Device Service will often establish at least two Addressable objects with the Core Metadata micro service. An Addressable is a flexible EdgeX object that specifies a physical address of something - in this case the physical address of the Device Service and the Device (the Modbus Device). While an Addressable could be created for a named MQTT pipe or other protocol endpoint, for this example, we will assume that both the Device Service and Device are able to be reached via HTTP REST calls.

So in this case, the Device Service would make two calls to Core Metadata, to create the Addressable for the Device Service:

In [ ]:
url = "http://localhost:48081/api/v1/addressable"

payload = "{\n\t\"name\":\"Camera Control\",\n\t\"protocol\":\"HTTP\",\n\t\"address\":\"172.17.0.1\",\n\t\"port\":49977,\n\t\"path\":\"/cameracontrol\",\n\t\"publisher\":\"none\",\n\t\"user\":\"none\",\n\t\"password\":\"none\",\n\t\"topic\":\"none\"\n}"
headers = {
    'cache-control': "no-cache"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

In [ ]:
import requests

url = "http://localhost:48081/api/v1/addressable"

payload = "{\n\t\"name\":\"camera1 address\",\n\t\"protocol\":\"HTTP\",\n\t\"address\":\"172.17.0.1\",\n\t\"port\":49999,\n\t\"path\":\"/camera1\",\n\t\"publisher\":\"none\",\n\t\"user\":\"none\",\n\t\"password\":\"none\",\n\t\"topic\":\"none\"\n}"
headers = { 'cache-control': "no-cache" }
response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

## Upload the YAML Profile

The YAML profile defines the device, device resources including all the Modbus registers, the HTTP addressable resources and RESTful URL commands that are used to read and write the resources. When you are done setting this up you will have a Modbus device that can have its value requested over and http request.


In [ ]:
import requests

url = "http://localhost:48081/api/v1/deviceprofile/uploadfile"

payload = "------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"file\"; filename=\"MBUS_RTH_LCD.yaml\"\r\nContent-Type: text/yaml\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW--"
headers = {
    'content-type': "multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW",
    'cache-control': "no-cache",
    'Postman-Token': "ee27e7b3-e951-41c5-9d90-f3c288aa6323"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

# Using the Modbus Device
## Verify the Device was Added

In [13]:
# Run this command in a terminal
sudo docker logs -f --tail 100 edgex-device-modbus

SyntaxError: invalid syntax (<ipython-input-13-20f811204d9e>, line 2)

## Get the ID or Device Name

Now that the device is defined within EdgeX, the next step is to get the device ID or name and use that to create a scheduled event that will read from the device.

To get a list of all devices you can issue a **GET** command to this URL.


In [14]:
import requests
import json

url = "http://localhost:48081/api/v1/device"

headers = {
    'cache-control': "no-cache",
    'Postman-Token': "ab72ff38-73a7-4364-b160-c12c33150110"
    }

response = requests.request("GET", url, headers=headers)

d = json.loads(response.text)
print(json.dumps(d, indent=4))

[
    {
        "created": 1545325427070,
        "modified": 1545325427070,
        "origin": 1545325427003,
        "description": "Auto-generate this virtual device. KMC BAC-121036CE BACnet thermostat",
        "id": "5c1bcb739f8fc20001b11d82",
        "name": "KMC.BAC-121036CE01",
        "adminState": "UNLOCKED",
        "operatingState": "ENABLED",
        "addressable": {
            "created": 1545325426995,
            "modified": 0,
            "origin": 1545325426966,
            "id": "5c1bcb729f8fc20001b11d81",
            "name": "KMC.BAC-121036CE01-virtual-addressable",
            "protocol": "OTHER",
            "method": "POST",
            "address": "edgex-device-virtual",
            "port": 49990,
            "path": null,
            "publisher": null,
            "user": null,
            "password": null,
            "topic": null,
            "baseURL": "OTHER://edgex-device-virtual:49990",
            "url": "OTHER://edgex-device-virtual:49990"
        },
   

The name is 'mbus-rth-lcd-device'

## Use the Name or ID to Create an Addressable

## Create a Schedule

## Create an Event that Calls the Schedule

# Extra Material
## Define the value descriptors

In [ ]:
# First, list all the value descriptors

import requests

url = "http://localhost:48080/api/v1/valuedescriptor"

payload = ""
headers = { 'cache-control': "no-cache" }
response = requests.request("GET", url, data=payload, headers=headers)

pjson(response.text)

In [ ]:
# Create the ValueDescriptor for the "humancounter"
import requests

url = "http://localhost:48080/api/v1/valuedescriptor"

payload = "{\n\t\"name\":\"humancount\",\n\t\"description\":\"people count\", \n\t\"min\":\"0\",\n\t\"max\":\"100\",\n\t\"type\":\"I\",\n\t\"uomLabel\":\"count\",\n\t\"defaultValue\":\"0\",\n\t\"formatting\":\"%s\",\n\t\"labels\":[\"count\",\"humans\"]\n}"
headers = { 'cache-control': "no-cache" }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

In [ ]:
# Create the ValueDescriptor for the "dogcounter"
import requests

url = "http://localhost:48080/api/v1/valuedescriptor"

payload = "{\n    \"defaultValue\": \"0\",\n    \"description\": \"dog count\",\n    \"formatting\": \"%s\",\n    \"labels\": [\n        \"count\",\n        \"canines\"\n    ],\n    \"max\": \"100\",\n    \"min\": \"0\",\n    \"name\": \"caninecount\",\n    \"type\": \"I\",\n    \"uomLabel\": \"count\"\n}"
headers = {
    'cache-control': "no-cache",
    'Postman-Token': "30b43a6e-14b1-46fb-9cd7-2fc6ce7e0f4d"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

In [ ]:
# Create the ValueDescriptor for the "depth"
import requests

url = "http://localhost:48080/api/v1/valuedescriptor"

payload = "{\n    \"defaultValue\": \"1\",\n    \"description\": \"scan distance\",\n    \"formatting\": \"%s\",\n    \"labels\": [\n        \"scan\",\n        \"distance\"\n    ],\n    \"max\": \"10\",\n    \"min\": \"1\",\n    \"name\": \"depth\",\n    \"type\": \"I\",\n    \"uomLabel\": \"feet\"\n}"
headers = {
    'cache-control': "no-cache",
    'Postman-Token': "04c48848-b798-40a4-8ab8-66559575955b"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

In [ ]:
# Create the ValueDescriptor for the "duration"
import requests

url = "http://localhost:48080/api/v1/valuedescriptor"

payload = "{\n\t\"name\":\"duration\",\n\t\"description\":\"time between events\", \n\t\"min\":\"10\",\n\t\"max\":\"180\",\n\t\"type\":\"I\",\n\t\"uomLabel\":\"seconds\",\n\t\"defaultValue\":\"10\",\n\t\"formatting\":\"%s\",\n\t\"labels\":[\"duration\",\"time\"]\n}\n"
headers = {
    'cache-control': "no-cache",
    'Postman-Token': "26dc8757-f88e-45f4-a45e-eead6ad88478"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

In [ ]:
# Create the ValueDescriptor for the "error"
import requests

url = "http://localhost:48080/api/v1/valuedescriptor"

payload = "{\n    \"defaultValue\": \"error\",\n    \"description\": \"error response message from a camera\",\n    \"formatting\": \"%s\",\n    \"labels\": [\n        \"error\",\n        \"message\"\n    ],\n    \"max\": \"\",\n    \"min\": \"\",\n    \"name\": \"cameraerror\",\n    \"type\": \"S\",\n    \"uomLabel\": \"\"\n}"
headers = {
    'cache-control': "no-cache",
    'Postman-Token': "c0c86617-67d4-4d98-bd44-657db122f10e"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

## Define the Device

In [16]:
import requests

url = "http://localhost:48081/api/v1/device"

headers = {
    'cache-control': "no-cache",
    'Postman-Token': "457b9fa6-df54-4575-9435-2ba7b6078815"
    }

response = requests.request("GET", url, headers=headers)

print(response.text)

[{"created":1545325427070,"modified":1545325427070,"origin":1545325427003,"description":"Auto-generate this virtual device. KMC BAC-121036CE BACnet thermostat","id":"5c1bcb739f8fc20001b11d82","name":"KMC.BAC-121036CE01","adminState":"UNLOCKED","operatingState":"ENABLED","addressable":{"created":1545325426995,"modified":0,"origin":1545325426966,"id":"5c1bcb729f8fc20001b11d81","name":"KMC.BAC-121036CE01-virtual-addressable","protocol":"OTHER","method":"POST","address":"edgex-device-virtual","port":49990,"path":null,"publisher":null,"user":null,"password":null,"topic":null,"baseURL":"OTHER://edgex-device-virtual:49990","url":"OTHER://edgex-device-virtual:49990"},"lastConnected":0,"lastReported":0,"labels":["thermostat","industrial"],"location":null,"service":{"created":1545325422994,"modified":1545325422994,"origin":1545325422954,"description":"","id":"5c1bcb6e9f8fc20001b11d74","name":"edgex-device-virtual","lastConnected":0,"lastReported":0,"operatingState":"ENABLED","labels":["virtual"]

### Writing Data to the Core Data Service